In [ ]:
import os
import numpy as np
import pandas as pd
import datetime as dt
from collections import Counter

In [ ]:
def read_csv(file_path):
    ds = pd.read_csv(file_path, header=6)
    ds.Date_Time = pd.to_datetime(ds.Date_Time, errors = 'coerce')
    ds = ds.set_index('Date_Time')
    ds.index = ds.index.tz_localize(None)
    ds = ds.sort_index()
    ds = ds[ds.index.notna()]
    return ds

def start_end_storm(rain_data, event_date):
    
    timeseries_event_date = rain_data.precip_in[(rain_data.index >= event_date) & (rain_data.index <= event_date + dt.timedelta(days=1))]
 
    rain_event_indices = timeseries_event_date[timeseries_event_date > 0].index
    
    if rain_event_indices.empty:
        # if no rain on event day, return early (no storm detected)
        return None, None, None
    
    # first time and last time it rained during day of event
    
    first_rain_datetime = rain_event_indices[0]
    last_rain_datetime = rain_event_indices[-1]
    
    #starts computing for datetimes before the first time it rained on day of event
    
    current_datetime = first_rain_datetime

    while True:
        # Check the 8-hour block before the first datetime for rain 
        block_start_datetime = current_datetime - pd.Timedelta(hours=8)
        block_end_datetime = current_datetime
        block_data = rain_data[(rain_data.index >= block_start_datetime) & (rain_data.index < block_end_datetime)]
        #print(block_data)
    
        if block_data.precip_in.sum() > 0:
            # If there was rain in the block, update current_datetime
            current_datetime = block_data[block_data.precip_in > 0].index[0]
        else:
        # If no rain in the block, break the loop
            break
        
    first_rain_datetime = current_datetime
     
#__________________________

    #starts computing for datetimes after last time it rained on day of event
   
    current_datetime = last_rain_datetime
    
    while True:
        # Check the 8-hour block after the last datetime for rain
        block_end_datetime = current_datetime + pd.Timedelta(hours=8)
        block_start_datetime = current_datetime
        block_data = rain_data[(rain_data.index > block_start_datetime) & (rain_data.index <= block_end_datetime)]
    
    #print(block_data)
    
        if block_data.precip_in.sum() > 0:
            # If there was rain in the block, update current_datetime
            current_datetime = block_data[block_data.precip_in > 0].index[-1]
        else:
        # If no rain in the block, break the loop
            break
            
    last_rain_datetime = current_datetime
    
    filtered_rainstorm_data = rain_data[(rain_data.index >= first_rain_datetime ) & (rain_data.index <= last_rain_datetime)]
    
    return first_rain_datetime, last_rain_datetime, filtered_rainstorm_data

def compute_duration_accummrain_avgintensity_mm_h(rainstorm_data):
    

    accumulated_rain = rainstorm_data.precip_in.sum()*25.4
    duration = (rainstorm_data.index[-1] - rainstorm_data.index[0]).total_seconds() / 3600
    average_intensity = accumulated_rain/duration
    
    return accumulated_rain, duration, average_intensity

def compute_peakintensity_mm(rainstorm_data, frequency_minutes):
    
    #compute the min frequency in the rain station file to know whether peak intensity can be calculated for frequency_minutes (e.g., 15-min)
    
    min_frequency = pd.tseries.frequencies.to_offset(rainstorm_data.index.to_series().diff().min())
    frequency = str(frequency_minutes)+'T'
    
    if min_frequency <= pd.tseries.frequencies.to_offset(frequency):
        
        max_acummrain = rainstorm_data.precip_in.rolling(frequency).sum().max()
        peak_intensity = (max_acummrain *60)*25.4/frequency_minutes
        datetime_peak_int= (rainstorm_data.precip_in.rolling(frequency).sum()).idxmax()
        
        return max_acummrain,peak_intensity,datetime_peak_int
        
    else:
        print('Rain data frequency is' + str(min_frequency) + 'Cannot compute peak intensity')
        return None, None, None
    
def compute_cummprecipitation(rain_data, storm_start, period_hours):
    
    #this computes antecedent rainfall for periods before the start of the storm associated with PFDF event
    #e.g. 24 hours before a storm that triggered PFDF started
    
    past_datetime = storm_start - dt.timedelta(hours = period_hours)
    acumm_rain = (rain_data.precip_in[(rain_data.index >= past_datetime) & (rain_data.index <= storm_start)]).sum() *25.4
    print(past_datetime, acumm_rain)
    
    return acumm_rain

def find_matching_csv(events, csv_folder_path):
    
    #iterate over rain station files
    for elem in os.listdir(csv_folder_path):
        file = os.path.join(csv_folder_path, elem)
        print(file)
        ds = read_csv(file)
        station_id = ds.Station_ID[0]
        print(station_id)
        
        #iterate over PFDF events
        for i in range(len(events)):
            event_station_id = events['Station ID'][i]
            #print(type(event_station_id))
            events.EVENT_DATE = pd.to_datetime(events.EVENT_DATE)
            event_date = events['EVENT_DATE'][i]

            # if the sation id matches and the PFDF event date is in the rain station file
            if (event_station_id == str(station_id)) and (event_date.strftime('%Y%m%d') in ds.index.strftime('%Y%m%d').unique()):
                print(f'Matching CSV found for Station ID {event_station_id} and Event Date {event_date}', i)
                
                #compute rainfall variables
                first_rain_datetime, last_rain_datetime, filtered_rainstorm_data = start_end_storm(ds, event_date)
                accumulated_rain, duration, average_intensity = compute_duration_accummrain_avgintensity_mm_h(filtered_rainstorm_data)
                max_acummrain15m,peak_int15m, datetime_peak_int15m = compute_peakintensity_mm(filtered_rainstorm_data,15)
                max_acummrain30m,peak_int30m,datetime_peak_int30m = compute_peakintensity_mm(filtered_rainstorm_data,30)
                max_acummrain60m,peak_int60m,datetime_peak_int60m = compute_peakintensity_mm(filtered_rainstorm_data,60)
                cumm_rain_24h = compute_cummprecipitation(ds, first_rain_datetime, 24)
                cumm_rain_48h = compute_cummprecipitation(ds, first_rain_datetime, 48)
                #print(peakint15m)
                
                #add results to the inventory dataframe
                events.loc[i, 'Storm_StartDate'] = first_rain_datetime
                events.loc[i, 'Storm_Duration'] = duration
                events.loc[i, 'Storm_Accum'] = accumulated_rain
                events.loc[i,'Storm_AvgIntensity']= average_intensity
                events.loc[i, 'Peak_I15_mm/h'] = peak_int15m
                events.loc[i, 'Peak_I30_mm/h'] = peak_int30m
                events.loc[i, 'Peak_I60_mm/h'] = peak_int60m
                events.loc[i, 'Peak_I15_Date'] = datetime_peak_int15m
                events.loc[i, 'Peak_I30_Date'] = datetime_peak_int30m
                events.loc[i, 'Peak_I60_Date'] = datetime_peak_int60m
                events.loc[i, 'CummRain_24h'] = cumm_rain_24h
                events.loc[i, 'CummRain_48h'] = cumm_rain_48h
                

                print(first_rain_datetime, accumulated_rain, duration, average_intensity, cumm_rain_24h, cumm_rain_48h)
                #print(rainstorm_filtered)
                
    return events

events_data = pd.read_csv('~/DATA/Project2/NewInventory.csv')  #file with PFDF events
csv_folder_path = os.path.join(os.path.expanduser('~'), 'DATA/Project2/Precip_csv2/precip_csv') #directory where rain station files are stored

events_newdata = find_matching_csv(events_data, csv_folder_path)

In [ ]:
#save dataframe
pd.DataFrame.to_csv(events_newdata, '/home/silvana/DATA/Project2/Rainfall_Data_New.csv')